In [24]:
import pandas as pd
import re
import json
import os
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import displacy
#import en_core_web_sm

In [25]:
df = pd.read_csv("pt_question_answers_updated.csv")
df

,pt_post_id,pt_post_type_id,pt_accepted_answer_id,pt_creation_date,pt_score,pt_title,pt_body,pt_tags,pt_parent_id,context,pt_answer,question
0,34750268,1,34762233.0,2016-01-12T17:36:25.473,9,Extracting the top-k value-indices from a 1-D ...,<p>Given a 1-D tensor in Torch (<code>torch.Te...,<python><lua><pytorch><torch>,NaN,<p>Just loop through the tensor and run your c...,"<p>As of pull request <a href=""https://github....",Extracting the top-k value-indices from a 1-D ...
1,38543850,1,38676842.0,2016-07-23T16:15:43.967,40,How to Display Custom Images in Tensorboard (e...,"<p>The <a href=""https://github.com/tensorflow/...",<python><tensorflow><matplotlib><pytorch><tens...,NaN,<p>It is quite easy to do if you have the imag...,<p>It is quite easy to do if you have the imag...,How to Display Custom Images in Tensorboard (e...
2,41767005,1,43824857.0,2017-01-20T15:22:08.063,11,Python wheels: cp27mu not supported,"<p>I'm trying to install pytorch (<a href=""htt...",<python><linux><unicode><pytorch>,NaN,<p>This is exactly that. \nRecompile python un...,<p>This is exactly that. \nRecompile python un...,Python wheels: cp27mu not supported <p>I'm try...
3,41861354,1,54261158.0,2017-01-25T20:45:35.297,8,Loading Torch7 trained models (.t7) in PyTorch,<p>I am using Torch7 library for implementing ...,<python><lua><pytorch><torch><pre-trained-model>,NaN,<p>The correct function is <code>load_lua</cod...,<p>As of PyTorch 1.0 <code>torch.utils.seriali...,Loading Torch7 trained models (.t7) in PyTorch...
4,41924453,1,42054194.0,2017-01-29T18:31:24.687,65,PyTorch: How to use DataLoaders for custom Dat...,<p>How to make use of the <code>torch.utils.da...,<python><torch><pytorch>,NaN,"<p>Yes, that is possible. Just create the obje...","<p>Yes, that is possible. Just create the obje...",PyTorch: How to use DataLoaders for custom Dat...
...,...,...,...,...,...,...,...,...,...,...,...,...
10758,74612146,1,NaN,2022-11-29T09:54:30.430,0,Is it possible to perform quantization on dens...,<p>I have been trying to performing quantizati...,<machine-learning><pytorch><artificial-intelli...,NaN,<p>Here's how to do this on DenseNet169 from t...,<p>Here's how to do this on DenseNet169 from t...,NaN
10759,74637151,1,NaN,2022-12-01T05:08:37.150,1,"Why when the batch size increased, the epoch t...",<p>Epoch time means the time required to train...,<deep-learning><pytorch>,NaN,"<p>As you already noticed, there are many fact...","<p>As you already noticed, there are many fact...",NaN
10760,74642594,1,NaN,2022-12-01T13:23:27.277,0,Why does StableDiffusionPipeline return black ...,"<p>I am using the <a href=""https://github.com/...",<python><pytorch><apple-m1><huggingface-transf...,NaN,<p>Apparently it is indeed an Apple Silicon (M...,<p>Apparently it is indeed an Apple Silicon (M...,NaN
10761,74671399,1,NaN,2022-12-03T22:46:46.443,1,Locating tags in a string in PHP (with respect...,<p>I want to create a function that labels the...,<php><string><pytorch><label><italics>,NaN,<p>I think I've got something. How about this:...,<p>I think I've got something. How about this:...,NaN


In [26]:
## remove html

CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

df["context"] = df["context"].apply(lambda x: cleanhtml(x))
df["pt_answer"] = df["pt_answer"].apply(lambda x: cleanhtml(x))

df["context"] = df["context"].str.lower()
df["pt_answer"] = df["pt_answer"].str.lower()

In [27]:
df["text"] = "question: " + df["pt_title"] + "\n" + "answer: " + df["pt_answer"]

df = df[["text"]]
df

,text
0,question: Extracting the top-k value-indices f...
1,question: How to Display Custom Images in Tens...
2,question: Python wheels: cp27mu not supported\...
3,question: Loading Torch7 trained models (.t7) ...
4,question: PyTorch: How to use DataLoaders for ...
...,...
10758,question: Is it possible to perform quantizati...
10759,"question: Why when the batch size increased, t..."
10760,question: Why does StableDiffusionPipeline ret...
10761,question: Locating tags in a string in PHP (wi...


In [83]:
# ! pip install bertopic

## BERTopic

In [28]:
from bertopic import BERTopic

In [29]:
topic_model = BERTopic()

In [40]:
topics, probs = topic_model.fit_transform(df["text"].tolist())

In [42]:
topic_model.get_topic(0)

[('install', 0.03725731290943771),
 ('conda', 0.027772767991368397),
 ('version', 0.025413716360693592),
 ('cuda', 0.023241619000067035),
 ('pip', 0.02183774332802657),
 ('pytorch', 0.017217043004825704),
 ('python', 0.01600398025597553),
 ('installed', 0.01487589525766413),
 ('environment', 0.014654145129711037),
 ('torchvision', 0.012218126707432449)]

## getting similar topics for query

In [89]:
similar_topics, similarity = topic_model.find_topics("How do I check if PyTorch is using the GPU?", top_n=5)

In [90]:
similar_topics

[19, 33, 26, 14, 0]

In [91]:
similarity

[0.3374741802006825,
 0.2855981646211909,
 0.28135442524359855,
 0.2702904180213215,
 0.2469874208791374]

In [92]:
for i in range(len(similar_topics)):
    print(topic_model.get_topic(similar_topics[i]))
    print("\n\n")

[('gpu', 0.04467353942010262), ('device', 0.04428266828288102), ('cpu', 0.03232311553494966), ('cuda', 0.020559549812386883), ('move', 0.01681275332598251), ('on', 0.014964692138828119), ('torchcudaisavailable', 0.014178443504363652), ('if', 0.013870270749976647), ('else', 0.013798813270039312), ('na', 0.011972828323193843)]



[('device', 0.05893344536339523), ('cpu', 0.036378669977404465), ('on', 0.023941120127165656), ('tensors', 0.020260408160267477), ('gpu', 0.019552824387833418), ('cuda0', 0.018790079991273655), ('devices', 0.0186472652307531), ('cuda', 0.01826774166341191), ('expected', 0.01794840583014162), ('todevice', 0.016202277584049567)]



[('gpu', 0.034695525239471586), ('time', 0.022516553808134682), ('cpu', 0.021264241177781574), ('operations', 0.012033072724665271), ('than', 0.011737173268690353), ('01', 0.011268864464680898), ('on', 0.011074207851253058), ('slower', 0.010956562476492376), ('much', 0.010942223553543849), ('run', 0.010177469650720668)]



[('gpus', 0.0

## getting similar docs

In [95]:
topic_model.get_representative_docs(topic=similar_topics[0])

["question: pytorch when do I need to use `.to(device)` on a model or tensor?\nanswer: it is necessary to have both the model, and the data on the same device, either cpu or gpu, for the model to process data. data on cpu and model on gpu, or vice-versa, will result in a runtime error.\nyou can set a variable device to cuda if it's available, else it will be set to cpu, and then transfer data and model to device :\nimport torch\n\ndevice = 'cuda' if torch.cuda.is_available() else 'cpu'\nmodel.to(device)\ndata = data.to(device)\n\n",
 'question: Pytorch 0.4.0: There are three ways to create tensors on CUDA device. Is there some difference between them?\nanswer: all three methods worked for me. \n\nin 1 and 2, you create a tensor on cpu and then move it to gpu when you use .to(device) or .cuda(). they are the same here.\n\nhowever, when you use .to(device) method you can explicitly tell torch to move to specific gpu by setting device=torch.device("cuda:&lt;id&gt;"). with .cuda() you have